In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cleaning sample_data directory
!rm -r sample_data

In [ ]:
!ls drive/My\ Drive/touche-2022-prototyping

initial-retrieval-prototyping-and-analysis.ipynb
touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl
touche-task2-passages-version-002.jsonl


In [ ]:
# One Time Exection.
# extracting files and converting them to 'tar.gz' files.
# !gzip -d drive/My\ Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl.gz
# !gzip -d drive/My\ Drive/touche-2022-prototyping/touche-task2-passages-version-002.jsonl.gz

In [ ]:
# installing linux related stuff for pyserini
# removing error "ImportError: No module named '_swigfaiss"
# Reference Link: https://github.com/facebookresearch/faiss/issues/821
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (301 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty

In [ ]:
# installing important packages for analyzing the code.
!pip install jsonlines
!pip install pyserini
!pip install faiss

     |████████████████████████████████| 84.6 MB 113 kB/s 
     |████████████████████████████████| 2.0 MB 34.0 MB/s 
     |████████████████████████████████| 3.8 MB 24.8 MB/s 
     |████████████████████████████████| 1.1 MB 32.8 MB/s 
     |████████████████████████████████| 4.9 MB 30.2 MB/s 
     |████████████████████████████████| 13.5 MB 33.1 MB/s 
     |████████████████████████████████| 6.0 MB 20.4 MB/s 
     |████████████████████████████████| 1.2 MB 37.3 MB/s 
     |████████████████████████████████| 188 kB 39.1 MB/s 
     |████████████████████████████████| 653 kB 43.5 MB/s 
     |████████████████████████████████| 451 kB 35.2 MB/s 
     |████████████████████████████████| 181 kB 49.5 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 10.1 MB 36.0 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 6.5 MB 34.7 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |██████████

In [ ]:
# import statements
import pandas as pd
import jsonlines

In [ ]:
# original corpus
# corpus_list = list(jsonlines.open('drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002.jsonl'))

In [ ]:
# query expansion corpus
qcorpus_list = list(jsonlines.open('drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl'))

In [ ]:
# printing demo query
qcorpus_list[0]

{'chatNoirUrl': 'https://chatnoir.eu/cache?uuid=25e04d49-8df7-58c9-8fae-c5bd54a070ba&index=cw12&raw&plain',
 'contents': "Do Asian-Americans Face Bias in Admissions at Elite Colleges? - NYTimes.com Home Page Today's Paper Video Most Popular Times Topics Search All NYTimes.com Education World U.S. Politics Education Bay Area Chicago Texas N.Y. / Region Business Technology Science Health Sports Opinion Arts Style Travel Jobs Real Estate Autos February 8, 2012, 1:43 pm Do Asian-Americans Face Bias in Admissions at Elite Colleges? By DANIEL E. SLOTNIK 6:08 p.m. | Updated A statement from Princeton was added to the story. The Department of Education’s Office for Civil Rights is examining complaints thatHarvard and Princeton have discriminated against Asian-American undergraduate applicants, highlighting a concern of many Asian-American parents. The inquiry was first reported by Bloomberg News.<query> do asian americans face bias in admissions</query>",
 'id': 'clueweb12-0000tw-00-14115___1'

In [ ]:
!mkdir sample_collection_jsonl
!mkdir indexes
!mkdir indexes/sample_collection_jsonl

In [ ]:
output = jsonlines.open('sample_collection_jsonl/documents.jsonl', 'w')
for vals in qcorpus_list:
    output.write({
        'id': vals['id'],
        'contents': vals['contents']
    })

In [ ]:
!head -10 sample_collection_jsonl/documents.jsonl

{"id": "clueweb12-0000tw-00-14115___1", "contents": "Do Asian-Americans Face Bias in Admissions at Elite Colleges? - NYTimes.com Home Page Today's Paper Video Most Popular Times Topics Search All NYTimes.com Education World U.S. Politics Education Bay Area Chicago Texas N.Y. / Region Business Technology Science Health Sports Opinion Arts Style Travel Jobs Real Estate Autos February 8, 2012, 1:43 pm Do Asian-Americans Face Bias in Admissions at Elite Colleges? By DANIEL E. SLOTNIK 6:08 p.m. | Updated A statement from Princeton was added to the story. The Department of Education’s Office for Civil Rights is examining complaints thatHarvard and Princeton have discriminated against Asian-American undergraduate applicants, highlighting a concern of many Asian-American parents. The inquiry was first reported by Bloomberg News.<query> do asian americans face bias in admissions</query>"}
{"id": "clueweb12-0000tw-00-14115___10", "contents": "By insisting on upholding affirmative action in colle

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/sample_collection_jsonl \
  --index /content/indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 4 \
  --storePositions --storeDocvectors --storeRaw

2022-03-18 02:24:52,355 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2022-03-18 02:24:52,358 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-03-18 02:24:52,358 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-03-18 02:24:52,359 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: /content/sample_collection_jsonl
2022-03-18 02:24:52,359 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-03-18 02:24:52,359 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-03-18 02:24:52,360 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 4
2022-03-18 02:24:52,366 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-03-18 02:24:52,367 INFO  [main] index.IndexC

In [ ]:
!ls /content/indexes/sample_collection_jsonl

_0.fdm	_0_Lucene80_0.dvd  _0_Lucene84_0.tim  _0.nvm  _0.tvx
_0.fdt	_0_Lucene80_0.dvm  _0_Lucene84_0.tip  _0.si   segments_1
_0.fdx	_0_Lucene84_0.doc  _0_Lucene84_0.tmd  _0.tvd  write.lock
_0.fnm	_0_Lucene84_0.pos  _0.nvd	      _0.tvm


In [ ]:
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher('indexes/sample_collection_jsonl')
hits = searcher.search('Who is stronger, Hulk or Superman?', k=12)
# Value of K determines the number of documents that can be returned.
print('Number of document matches from the index: '+str(len(hits)))
# Attempted tuning BM25 and RM3, nothing has changed the document ranking below.
for i in range(len(hits)):
    print(f'{i+1} {hits[i].docid} {hits[i].score:.5f}')

Number of document matches from the index: 12
1 clueweb12-1400wb-06-30596___3 16.55650
2 clueweb12-0605wb-29-30060___11 15.76430
3 clueweb12-1709wb-12-07226___7 15.69540
4 clueweb12-0605wb-29-30060___2 15.21990
5 clueweb12-0904wb-33-14589___7 15.14270
6 clueweb12-1400wb-06-30596___4 15.02100
7 clueweb12-1411wb-25-14634___8 14.83110
8 clueweb12-0605wb-29-30060___10 14.75490
9 clueweb12-1014wb-84-07457___12 14.68790
10 clueweb12-1400tw-67-08199___28 14.26160
11 clueweb12-0306wb-63-29079___5 14.16490
12 clueweb12-0605wb-57-17257___8 13.69220


In [ ]:
# Grab the raw text:
print(hits[0].raw)
print('\n')
# Grab the raw Lucene Document:
print(hits[0].lucene_document)

{
  "id" : "clueweb12-1400wb-06-30596___3",
  "contents" : "There is a limit to how angry someone gets...even the Hulk and even at his angriest he wouldn't be as strong as Superman. No one really knows how strong the man of steel is and perhaps neither does he! Many wrongly think that just because Hulk is bigger and looks stronger than it means he is stronger. Superman can move planets...need I say more! Others believe Hulk would win because they're likely Marvel fan boys. Superman has defeated powerful opponents like Darkseid,Mongrel,Parasite,Doomsday,and General Zod;several of whom could likely take Hulk! Superman has more varied powers and because he's smarter can be more resourceful in using them. Speaking of intelligence,how the hell can anyone think that Superman isn't smarter than a human,even one as brilliant as Dr.Bruce Banner. Superman/Kal-El is not only the son of Jor-El,Krypton's most brilliant scientist,but he's outsmarted Lex Luthor(one of the smartest people on earth)and

In [ ]:
# Building Dense Index, GPU activated for building dense index.
# This index is build to do "Dense" and "Sparse" retrieval from documents.

In [ ]:
!mkdir indexes/sample_collection_jsonl_dense

In [ ]:
# Encoding documents with dense encoder representations
# Expected time was showing around 15 hours
# Sample execution is checked, but this should be prepared after more analysis.
# Colab Pro, is definitely required for preparing the dense representations.
# Sharding approach doesn't execute indexes in parallel. Hence, only shard num = 1 allowed.
!python -m pyserini.encode input --corpus /content/sample_collection_jsonl \
          --fields text \
          --delimiter "\n" \
          --shard-id 0 \
          --shard-num 1 \
  output  --embeddings /content/indexes/sample_collection_jsonl_dense \
          --to-faiss \
  encoder --encoder castorini/tct_colbert-v2-hnp-msmarco \
          --fields text \
          --batch 32 \
          --fp16

868654it [00:08, 99470.32it/s]
  0% 16/13573 [00:45<10:36:15,  2.82s/it]
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/pyserini/encode/__main__.py", line 103, in <module>
    embeddings = encoder.encode(**kwargs)
  File "/usr/local/lib/python3.7/dist-packages/pyserini/encode/_tct_colbert.py", line 66, in encode
    outputs = self.model(**inputs)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/transformers/models/bert/modeling_bert.py", line 1006, in forward
    return_dict=return_dict,
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  F

In [ ]:
# Encoding attempt with sparse encoder
# Expected time for completion is even more for these encoders.
!python -m pyserini.encode \
  input   --corpus /content/sample_collection_jsonl \
          --fields text \
  output  --embeddings /content/indexes/sample_collection_jsonl_dense \
  encoder --encoder castorini/unicoil-msmarco-passage \
          --fields text \
          --batch 32 \
          --fp16

Downloading: 100% 677/677 [00:00<00:00, 1.01MB/s]
Downloading: 100% 418M/418M [00:12<00:00, 35.9MB/s]
Downloading: 100% 226k/226k [00:00<00:00, 1.99MB/s]
Downloading: 100% 112/112 [00:00<00:00, 95.2kB/s]
Downloading: 100% 300/300 [00:00<00:00, 311kB/s]
868654it [00:09, 92140.35it/s]
  0% 7/27146 [00:25<27:01:00,  3.58s/it]
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/pyserini/encode/__main__.py", line 103, in <module>
    embeddings = encoder.encode(**kwargs)
  File "/usr/local/lib/python3.7/dist-packages/pyserini/encode/_unicoil.py", line 96, in encode
    batch_weights = self.model(input_ids).cpu().detach().numpy()
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/u

In [ ]:
# Finally, converting it into HNSW index for retrieval.
# Getting Dense and Hybrid retrieval results, code available in README.md